<a href="https://colab.research.google.com/github/314sirius/linked-in/blob/main/MyPosts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import seaborn as sns
#import emoji

from wordcloud import WordCloud,STOPWORDS

In [ ]:
from google.colab import files

In [ ]:
# Upload your Shares.csv file here
uploaded=files.upload()

In [ ]:
df = pd.read_csv('Shares.csv', encoding = 'utf-8')
df

In [ ]:
!pip install transformers
!pip install tqdm

import pandas as pd
import re
from transformers import pipeline
from tqdm import tqdm

# Assuming df and the 'Message_no_link' column are already defined as in your code

# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Function to analyze sentiment with progress bar
def analyze_sentiment_with_progress(messages):
    sentiments = []
    for message in tqdm(messages, desc="Analyzing sentiment"):
      try:
        sentiments.append(sentiment_pipeline(message)[0])
      except Exception as e:
        print(f"Error analyzing: {message}. Error:{e}")
        sentiments.append({'label':'ERROR', 'score':0})
    return sentiments

# Analyze sentiment with progress bar
#df_filtered['Sentiment'] = analyze_sentiment_with_progress(df['Cleaned_Post'].astype(str))
#df_filtered

sentiment_results = analyze_sentiment_with_progress(df['ShareCommentary'].astype(str))

df[['Sentiment_Label', 'Sentiment_Score']] = pd.DataFrame(
    [(result['label'], result['score']) for result in sentiment_results],
    index=df.index
)

In [ ]:
df

In [ ]:
# prompt: create new comumn relative_score if sentiment label is negative than sentiment_score*(-1) else sentiment_score, in one line

df['relative_score'] = df.apply(lambda row: row['Sentiment_Score'] * (-1) if row['Sentiment_Label'] == 'NEGATIVE' else row['Sentiment_Score'], axis=1)


In [ ]:
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_filtered is already created as in your previous code

# Convert 'Date' to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])
df['Month-Year'] = df['Date'].dt.to_period('M')

# Create the heatmap
plt.figure(figsize=(12, 8))

# Create a pivot table for the heatmap data using 'Month-Year'
heatmap_data = df.pivot_table(index=df['Date'].dt.day,
                                       columns=df['Month-Year'],
                                       values='relative_score', aggfunc='mean') # Use mean aggregation



# Create custom colormap for positive (green) and negative (red) sentiments
import matplotlib.colors as mcolors


# Use custom colormap to determine colors and intensity based on sentiment scores
sns.heatmap(heatmap_data, cmap="viridis", annot=True, fmt=".2f", linewidths=0.5,
            cbar_kws={'label': 'Sentiment Score'},
            center=0, linecolor='darkgrey', alpha=0.7)

plt.style.use('dark_background')

plt.yticks(rotation=0, ha='right')

plt.title('Sentiment Heatmap: Day vs. Month')
plt.xlabel('Month')
plt.ylabel('Day')
plt.show()

In [ ]:
# prompt: preprocess the text in ShareCommentary to sentiment analysis

def preprocess_text(text):
    """Preprocesses text for sentiment analysis."""
    if not isinstance(text, str):  # Handle non-string inputs
        return ""

    # Lowercasing
    text = text.lower()

    # Removing URLs
    text = re.sub(r'http\S+', '', text)

    # Removing mentions (@usernames)
    text = re.sub(r'@\w+', '', text)

    # Removing hashtags (#hashtags) -  Decide if you want to keep or remove
    text = re.sub(r'#\w+', '', text)

    # Removing special characters and punctuation (except for spaces)
    text = re.sub(r'[^\w\s]', '', text)

    # Removing extra whitespace
    text = ' '.join(text.split())

    return text


# Apply the preprocessing function to the 'ShareCommentary' column
df['Cleaned_ShareCommentary'] = df['ShareCommentary'].apply(preprocess_text)

# Now use 'Cleaned_ShareCommentary' for sentiment analysis
sentiment_results = analyze_sentiment_with_progress(df['Cleaned_ShareCommentary'].astype(str))

df[['Sentiment_Label', 'Sentiment_Score']] = pd.DataFrame(
    [(result['label'], result['score']) for result in sentiment_results],
    index=df.index
)


In [ ]:
df['Length'] = df['Cleaned_ShareCommentary'].str.len().fillna(0).astype(int)

In [ ]:
# Convert 'Date' column to datetime, dropping rows with invalid dates
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])


In [ ]:
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.plot(df['Date'], df['Length'])
plt.xlabel('Date')
plt.ylabel('Length')
plt.title('Comment Length Over Time')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()

In [ ]:
#calculate daily comment count
monthly_post_count = df.groupby(pd.Grouper(key='Date', freq='m'))['Cleaned_ShareCommentary'].count()

# Print the result
monthly_post_count

In [ ]:
df.info()

In [ ]:
pip install emoji

In [ ]:
# count emoji in comments
import emoji

def count_emoji(text):
  #text=str(text)
  emoji_count = 0
  for char in text:
    if emoji.is_emoji(char):
      emoji_count += 1
  return emoji_count


df['ShareCommentary'] = df['ShareCommentary'].astype(str)

df['emoji_number'] = df['ShareCommentary'].apply(count_emoji)

df

In [ ]:
total_length_posts=df['Length'].sum()
total_length_posts